# 第九章：从零开始构建复杂提示词

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

恭喜您来到最后一章！现在是时候将所有内容整合起来，学习如何**创建独特而复杂的提示词**。

下面，您将使用我们**为复杂提示词推荐的指导性结构**。在本章的后续部分，我们将向您展示一些行业特定的提示词，并解释这些提示词是如何采用类似结构的。

**注意：** **并非所有提示词都需要以下复杂结构的每个元素**。我们鼓励您试验包含或排除某些元素，看看这如何影响模型的响应。通常**最好先使用许多提示词元素让您的提示词正常工作，然后再进行精炼和简化**。

### 示例 - 职业教练聊天机器人

以下结构结合了多种提示词工程元素，是复杂提示词的良好起点。**对于某些元素，顺序很重要**，对于其他元素则不然。我们会在最佳实践表明顺序重要时进行说明，但总的来说，**如果您按照这个顺序，将为出色的提示词奠定良好基础**。

对于以下示例，我们将构建一个受控角色扮演的提示词，其中模型扮演具有特定任务的情境角色。我们的目标是提示模型扮演一个友好的职业教练。

阅读然后运行下面的单元格，将各种提示词元素编译成一个完整的提示词。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 对话历史（这也可以在API调用中作为之前的`user`和`assistant`消息添加）
HISTORY = """客户：请给我两个社会学专业可能的职业选择。

小智：以下是社会学专业的两个潜在职业：

- 社会工作者 - 社会学为理解人类行为和社会系统提供了坚实的基础。通过额外的培训或认证，社会学学位可以让毕业生胜任社会工作者、案例管理员、顾问和帮助个人和团体的社区组织者等角色。

- 人力资源专员 - 从社会学中获得的群体动力学和组织行为学理解适用于人力资源职业。毕业生可能在招聘、员工关系、培训和发展、多样性和包容性以及其他HR职能中找到角色。对社会结构和机构的关注也支持公共政策、非营利管理和教育等相关职业。"""

# 第二个输入变量 - 用户的问题
QUESTION = "这两个职业中哪一个需要超过学士学位的学历？"



######################################## 提示词元素 ########################################

##### 提示词元素 1: `user` 角色
# 确保您的消息API调用始终在消息数组中以 `user` 角色开始。
# 上面定义的 get_completion() 函数会自动为您执行此操作。

##### 提示词元素 2: 任务上下文
# 给模型关于它应该承担的角色或您希望它通过提示词承担的目标和总体任务的上下文。
# 最好在提示词正文的早期放置上下文。
TASK_CONTEXT = "您将扮演由星途职业公司创建的名为小智的AI职业教练。您的目标是为用户提供职业建议。您将回复在星途网站上的用户，如果您不以小智的身份回应，他们会感到困惑。"

##### 提示词元素 3: 语调上下文
# 如果对交互很重要，告诉模型应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = "您应该保持友好的客户服务语调。"

##### 提示词元素 4: 详细任务描述和规则
# 详细说明您希望模型执行的具体任务，以及模型可能需要遵循的任何规则。
# 这也是您可以在模型没有答案或不知道时给它一个"出路"的地方。
# 最好将此描述和规则展示给朋友，以确保它布局合理，任何模糊的词语都得到清晰定义。
TASK_DESCRIPTION = """以下是交互的一些重要规则：
- 始终保持角色，作为小智，星途职业公司的AI
- 如果您不确定如何回应，请说"抱歉，我没有理解。您能重新表述一下您的问题吗？"
- 如果有人问不相关的问题，请说"抱歉，我是小智，我提供职业建议。您今天有什么职业问题我可以帮助您吗？\""""

##### 提示词元素 5: 示例
# 为模型提供至少一个可以模仿的理想响应示例。将其封装在 <example></example> XML 标签中。可以提供多个示例。
# 如果您确实提供多个示例，请给模型关于这是什么示例的上下文，并将每个示例封装在自己的XML标签集中。
# 示例可能是知识工作中让模型按预期行为的最有效工具。
# 确保为模型提供常见边缘情况的示例。如果您的提示词使用草稿本，提供草稿本应该如何看起来的示例是有效的。
# 通常更多示例 = 更好。
EXAMPLES = """以下是在标准交互中如何回应的示例：
<example>
客户：您好，您是如何被创建的，您做什么？
小智：您好！我的名字是小智，我由星途职业公司创建，提供职业建议。今天我可以为您提供什么帮助？
</example>"""

##### 提示词元素 6: 要处理的输入数据
# 如果提示词中有模型需要处理的数据，请在此处包含在相关的XML标签内。
# 可以包含多个数据片段，但务必将每个数据封装在自己的XML标签集中。
# 根据任务，此元素可能不是必需的。顺序也是灵活的。
INPUT_DATA = f"""以下是问题之前的对话历史（您和用户之间）。如果没有历史记录，可能为空：
<history>
{HISTORY}
</history>

以下是用户的问题：
<question>
{QUESTION}
</question>"""

##### 提示词元素 7: 立即任务描述或请求 #####
# "提醒"模型或准确告诉模型期望它立即做什么来完成提示词的任务。
# 这也是您放置额外变量（如用户问题）的地方。
# 向模型重申其立即任务通常不会有害。最好在长提示词末尾执行此操作。
# 这比在开头放置会产生更好的结果。
# 将用户查询放在提示词底部附近也是一般的良好做法。
IMMEDIATE_TASK = "您如何回应用户的问题？"

##### 提示词元素 8: 预认知（逐步思考）
# 对于有多个步骤的任务，最好告诉模型在给出答案之前先逐步思考
# 有时，您可能甚至必须说"在您给出答案之前..."以确保模型首先执行此操作。
# 并非所有提示词都需要，但如果包含，最好在长提示词末尾且在最终立即任务请求或描述之后执行此操作。
PRECOGNITION = "在回应之前先思考您的答案。"

##### 提示词元素 9: 输出格式
# 如果您希望模型的响应以特定方式格式化，请清楚地告诉模型该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果包含它，最好将其放在提示词末尾而不是开头。
OUTPUT_FORMATTING = "将您的响应放在 <response></response> 标签中。"

##### 提示词元素 10: 预填充模型的响应（如果有）
# 用一些预填充的词语开始模型的答案，以引导模型的行为或响应。
# 如果您想预填充模型的响应，您必须将其放在API调用的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = "[小智] <response>"



######################################## 组合元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示词
print("--------------------------- 包含变量替换的完整提示词 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)

现在让我们运行提示词！运行下面的单元格查看模型的输出。

In [ ]:
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

### 示例 - 法律服务

**法律专业内的提示词可能相当复杂**，因为需要：
- 解析长文档
- 处理复杂主题
- 以非常特定的方式格式化输出
- 遵循多步骤分析过程

让我们看看如何使用复杂提示词模板为特定法律用例构建提示词。下面，我们详细介绍了一个法律用例的示例提示词，其中我们要求模型使用法律文档中的信息回答关于法律问题的问题。

我们**更改了一些元素的顺序**以展示提示词结构可以是灵活的！

**提示词工程是关于科学的试验和错误**。我们鼓励您混合搭配，移动元素（顺序不重要的元素），看看什么最适合您和您的需求。 

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# 第二个输入变量 - 用户的问题
QUESTION = "是否有关于飓风期间如何处理宠物的法律？"



######################################## 提示词元素 ########################################

##### 提示词元素 1: `user` 角色
# 确保您的消息API调用始终在消息数组中以 `user` 角色开始。
# 上面定义的 get_completion() 函数会自动为您执行此操作。

##### 提示词元素 2: 任务上下文
# 给模型关于它应该承担的角色或您希望它通过提示词承担的目标和总体任务的上下文。
# 最好在提示词正文的早期放置上下文。
TASK_CONTEXT = "您是一位专业律师。"

##### 提示词元素 3: 语调上下文
# 如果对交互很重要，告诉模型应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = ""

##### 提示词元素 4: 要处理的输入数据
# 如果提示词中有模型需要处理的数据，请在此处包含在相关的XML标签内。
# 可以包含多个数据片段，但务必将每个数据封装在自己的XML标签集中。
# 根据任务，此元素可能不是必需的。顺序也是灵活的。
INPUT_DATA = f"""以下是已编译的一些研究资料。请使用它来回答用户的法律问题。
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### 提示词元素 5: 示例
# 为模型提供至少一个可以模仿的理想响应示例。将其封装在 <example></example> XML 标签中。可以提供多个示例。
# 如果您确实提供多个示例，请给模型关于这是什么示例的上下文，并将每个示例封装在自己的XML标签集中。
# 示例可能是知识工作中让模型按预期行为的最有效工具。
# 确保为模型提供常见边缘情况的示例。如果您的提示词使用草稿本，提供草稿本应该如何看起来的示例是有效的。
# 通常更多示例 = 更好。
EXAMPLES = """在您的答案中引用法律研究时，请使用包含搜索索引ID的方括号，后跟句号。将这些放在进行引用的句子末尾。正确引用格式的示例：

<examples>
<example>
此类犯罪的诉讼时效在10年后到期。[3]。
</example>
<example>
但是，当双方明确放弃时，保护不适用。[5]。
</example>
</examples>"""

##### 提示词元素 6: 详细任务描述和规则
# 详细说明您希望模型执行的具体任务，以及模型可能需要遵循的任何规则。
# 这也是您可以在模型没有答案或不知道时给它一个"出路"的地方。
# 最好将此描述和规则展示给朋友，以确保它布局合理，任何模糊的词语都得到清晰定义。
TASK_DESCRIPTION = """对以下问题写一个清晰、简洁的答案：

<question>
{QUESTION}
</question>

不应超过几段。如果可能，应以直接回答用户问题的单个句子结尾。但是，如果编译的研究中没有足够的信息来产生这样的答案，您可以推托并写"抱歉，我手头没有足够的信息来回答这个问题。\""""

##### 提示词元素 7: 立即任务描述或请求 #####
# "提醒"模型或准确告诉模型期望它立即做什么来完成提示词的任务。
# 这也是您放置额外变量（如用户问题）的地方。
# 向模型重申其立即任务通常不会有害。最好在长提示词末尾执行此操作。
# 这比在开头放置会产生更好的结果。
# 将用户查询放在提示词底部附近也是一般的良好做法。
IMMEDIATE_TASK = ""

##### 提示词元素 8: 预认知（逐步思考）
# 对于有多个步骤的任务，最好告诉模型在给出答案之前先逐步思考
# 有时，您可能甚至必须说"在您给出答案之前..."以确保模型首先执行此操作。
# 并非所有提示词都需要，但如果包含，最好在长提示词末尾且在最终立即任务请求或描述之后执行此操作。
PRECOGNITION = "在回答之前，请在 <relevant_quotes> 标签中提取研究中最相关的引用。"

##### 提示词元素 9: 输出格式
# 如果您希望模型的响应以特定方式格式化，请清楚地告诉模型该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果包含它，最好将其放在提示词末尾而不是开头。
OUTPUT_FORMATTING = "将您的两段响应放在 <answer> 标签中。"

##### 提示词元素 10: 预填充模型的响应（如果有）
# 用一些预填充的词语开始模型的答案，以引导模型的行为或响应。
# 如果您想预填充模型的响应，您必须将其放在API调用的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示词
print("--------------------------- 包含变量替换的完整提示词 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)

现在让我们运行提示词！运行下面的单元格查看模型的输出。

In [ ]:
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

如果您想在不更改上述任何内容的情况下试验课程提示词，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 9.1 - 金融服务聊天机器人](#exercise-91---financial-services-chatbot)
- [练习 9.2 - 代码机器人](#exercise-92---codebot)

### 练习 9.1 - 金融服务聊天机器人
金融专业内的提示词同样可能相当复杂，原因与法律提示词相似。这里是一个金融用例的练习，其中模型被用来**分析税务信息并回答问题**。就像法律服务示例一样，我们改变了一些元素的顺序，因为我们的解决方案提示词在不同的流程下更有意义（不过，其他结构也同样有效）。

我们建议您阅读变量内容（在这种情况下是 `{QUESTION}` 和 `{TAX_CODE}`）以了解模型预期要处理的内容。确保在您的提示词中的某个地方直接引用 `{QUESTION}` 和 `{TAX_CODE}`（使用与其他示例中相同的f-string语法），以便实际的变量内容可以被替换进去。

使用与描述和您在前面复杂提示词示例中看到的示例相匹配的内容填写提示词元素字段。一旦您填写完所有您想填写的提示词元素，运行单元格以查看连接的提示词以及模型的响应。

记住，提示词工程很少是纯粹公式化的，特别是对于大型和复杂的提示词！重要的是开发测试用例并**尝试各种提示词和提示词结构，看看什么对每种情况最有效**。请注意，如果您*确实*改变了提示词元素的顺序，您也应该记住在 `COMBINE ELEMENTS` 部分中改变连接的顺序。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 用户的问题
QUESTION = "我有多长时间来进行83b选择？"

# 第二个输入变量 - 模型将用来回答用户问题的税法文档
TAX_CODE = """
(a)General rule
If, in connection with the performance of services, property is transferred to any person other than the person for whom such services are performed, the excess of—
(1)the fair market value of such property (determined without regard to any restriction other than a restriction which by its terms will never lapse) at the first time the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, over
(2)the amount (if any) paid for such property,
shall be included in the gross income of the person who performed such services in the first taxable year in which the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever is applicable. The preceding sentence shall not apply if such person sells or otherwise disposes of such property in an arm’s length transaction before his rights in such property become transferable or not subject to a substantial risk of forfeiture.
(b)Election to include in gross income in year of transfer
(1)In general
Any person who performs services in connection with which property is transferred to any person may elect to include in his gross income for the taxable year in which such property is transferred, the excess of—
(A)the fair market value of such property at the time of transfer (determined without regard to any restriction other than a restriction which by its terms will never lapse), over
(B)the amount (if any) paid for such property.
If such election is made, subsection (a) shall not apply with respect to the transfer of such property, and if such property is subsequently forfeited, no deduction shall be allowed in respect of such forfeiture.
(2)Election
An election under paragraph (1) with respect to any transfer of property shall be made in such manner as the Secretary prescribes and shall be made not later than 30 days after the date of such transfer. Such election may not be revoked except with the consent of the Secretary.

(c)Special rules
For purposes of this section—
(1)Substantial risk of forfeiture
The rights of a person in property are subject to a substantial risk of forfeiture if such person’s rights to full enjoyment of such property are conditioned upon the future performance of substantial services by any individual.

(2)Transferability of property
The rights of a person in property are transferable only if the rights in such property of any transferee are not subject to a substantial risk of forfeiture.

(3)Sales which may give rise to suit under section 16(b) of the Securities Exchange Act of 1934
So long as the sale of property at a profit could subject a person to suit under section 16(b) of the Securities Exchange Act of 1934, such person’s rights in such property are—
(A)subject to a substantial risk of forfeiture, and
(B)not transferable.
(4)For purposes of determining an individual’s basis in property transferred in connection with the performance of services, rules similar to the rules of section 72(w) shall apply.
(d)Certain restrictions which will never lapse
(1)Valuation
In the case of property subject to a restriction which by its terms will never lapse, and which allows the transferee to sell such property only at a price determined under a formula, the price so determined shall be deemed to be the fair market value of the property unless established to the contrary by the Secretary, and the burden of proof shall be on the Secretary with respect to such value.

(2)Cancellation
If, in the case of property subject to a restriction which by its terms will never lapse, the restriction is canceled, then, unless the taxpayer establishes—
(A)that such cancellation was not compensatory, and
(B)that the person, if any, who would be allowed a deduction if the cancellation were treated as compensatory, will treat the transaction as not compensatory, as evidenced in such manner as the Secretary shall prescribe by regulations,
the excess of the fair market value of the property (computed without regard to the restrictions) at the time of cancellation over the sum of—
(C)the fair market value of such property (computed by taking the restriction into account) immediately before the cancellation, and
(D)the amount, if any, paid for the cancellation,
shall be treated as compensation for the taxable year in which such cancellation occurs.
(e)Applicability of section
This section shall not apply to—
(1)a transaction to which section 421 applies,
(2)a transfer to or from a trust described in section 401(a) or a transfer under an annuity plan which meets the requirements of section 404(a)(2),
(3)the transfer of an option without a readily ascertainable fair market value,
(4)the transfer of property pursuant to the exercise of an option with a readily ascertainable fair market value at the date of grant, or
(5)group-term life insurance to which section 79 applies.
(f)Holding period
In determining the period for which the taxpayer has held property to which subsection (a) applies, there shall be included only the period beginning at the first time his rights in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier.

(g)Certain exchanges
If property to which subsection (a) applies is exchanged for property subject to restrictions and conditions substantially similar to those to which the property given in such exchange was subject, and if section 354, 355, 356, or 1036 (or so much of section 1031 as relates to section 1036) applied to such exchange, or if such exchange was pursuant to the exercise of a conversion privilege—
(1)such exchange shall be disregarded for purposes of subsection (a), and
(2)the property received shall be treated as property to which subsection (a) applies.
(h)Deduction by employer
In the case of a transfer of property to which this section applies or a cancellation of a restriction described in subsection (d), there shall be allowed as a deduction under section 162, to the person for whom were performed the services in connection with which such property was transferred, an amount equal to the amount included under subsection (a), (b), or (d)(2) in the gross income of the person who performed such services. Such deduction shall be allowed for the taxable year of such person in which or with which ends the taxable year in which such amount is included in the gross income of the person who performed such services.

(i)Qualified equity grants
(1)In general
For purposes of this subtitle—
(A)Timing of inclusion
If qualified stock is transferred to a qualified employee who makes an election with respect to such stock under this subsection, subsection (a) shall be applied by including the amount determined under such subsection with respect to such stock in income of the employee in the taxable year determined under subparagraph (B) in lieu of the taxable year described in subsection (a).

(B)Taxable year determined
The taxable year determined under this subparagraph is the taxable year of the employee which includes the earliest of—
(i)the first date such qualified stock becomes transferable (including, solely for purposes of this clause, becoming transferable to the employer),
(ii)the date the employee first becomes an excluded employee,
(iii)the first date on which any stock of the corporation which issued the qualified stock becomes readily tradable on an established securities market (as determined by the Secretary, but not including any market unless such market is recognized as an established securities market by the Secretary for purposes of a provision of this title other than this subsection),
(iv)the date that is 5 years after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, or
(v)the date on which the employee revokes (at such time and in such manner as the Secretary provides) the election under this subsection with respect to such stock.
(2)Qualified stock
(A)In general
For purposes of this subsection, the term “qualified stock” means, with respect to any qualified employee, any stock in a corporation which is the employer of such employee, if—
(i)such stock is received—
(I)in connection with the exercise of an option, or
(II)in settlement of a restricted stock unit, and
(ii)such option or restricted stock unit was granted by the corporation—
(I)in connection with the performance of services as an employee, and
(II)during a calendar year in which such corporation was an eligible corporation.
(B)Limitation
The term “qualified stock” shall not include any stock if the employee may sell such stock to, or otherwise receive cash in lieu of stock from, the corporation at the time that the rights of the employee in such stock first become transferable or not subject to a substantial risk of forfeiture.

(C)Eligible corporation
For purposes of subparagraph (A)(ii)(II)—
(i)In general
The term “eligible corporation” means, with respect to any calendar year, any corporation if—
(I)no stock of such corporation (or any predecessor of such corporation) is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) during any preceding calendar year, and
(II)such corporation has a written plan under which, in such calendar year, not less than 80 percent of all employees who provide services to such corporation in the United States (or any possession of the United States) are granted stock options, or are granted restricted stock units, with the same rights and privileges to receive qualified stock.
(ii)Same rights and privileges
For purposes of clause (i)(II)—
(I)except as provided in subclauses (II) and (III), the determination of rights and privileges with respect to stock shall be made in a similar manner as under section 423(b)(5),
(II)employees shall not fail to be treated as having the same rights and privileges to receive qualified stock solely because the number of shares available to all employees is not equal in amount, so long as the number of shares available to each employee is more than a de minimis amount, and
(III)rights and privileges with respect to the exercise of an option shall not be treated as the same as rights and privileges with respect to the settlement of a restricted stock unit.
(iii)Employee
For purposes of clause (i)(II), the term “employee” shall not include any employee described in section 4980E(d)(4) or any excluded employee.

(iv)Special rule for calendar years before 2018
In the case of any calendar year beginning before January 1, 2018, clause (i)(II) shall be applied without regard to whether the rights and privileges with respect to the qualified stock are the same.

(3)Qualified employee; excluded employee
For purposes of this subsection—
(A)In general
The term “qualified employee” means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.
(B)Excluded employee
The term “excluded employee” means, with respect to any corporation, any individual—
(i)who is a 1-percent owner (within the meaning of section 416(i)(1)(B)(ii)) at any time during the calendar year or who was such a 1 percent owner at any time during the 10 preceding calendar years,
(ii)who is or has been at any prior time—
(I)the chief executive officer of such corporation or an individual acting in such a capacity, or
(II)the chief financial officer of such corporation or an individual acting in such a capacity,
(iii)who bears a relationship described in section 318(a)(1) to any individual described in subclause (I) or (II) of clause (ii), or
(iv)who is one of the 4 highest compensated officers of such corporation for the taxable year, or was one of the 4 highest compensated officers of such corporation for any of the 10 preceding taxable years, determined with respect to each such taxable year on the basis of the shareholder disclosure rules for compensation under the Securities Exchange Act of 1934 (as if such rules applied to such corporation).
(4)Election
(A)Time for making election
An election with respect to qualified stock shall be made under this subsection no later than 30 days after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, and shall be made in a manner similar to the manner in which an election is made under subsection (b).

(B)Limitations
No election may be made under this section with respect to any qualified stock if—
(i)the qualified employee has made an election under subsection (b) with respect to such qualified stock,
(ii)any stock of the corporation which issued the qualified stock is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) at any time before the election is made, or
(iii)such corporation purchased any of its outstanding stock in the calendar year preceding the calendar year which includes the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, unless—
(I)not less than 25 percent of the total dollar amount of the stock so purchased is deferral stock, and
(II)the determination of which individuals from whom deferral stock is purchased is made on a reasonable basis.
(C)Definitions and special rules related to limitation on stock redemptions
(i)Deferral stock
For purposes of this paragraph, the term “deferral stock” means stock with respect to which an election is in effect under this subsection.

(ii)Deferral stock with respect to any individual not taken into account if individual holds deferral stock with longer deferral period
Stock purchased by a corporation from any individual shall not be treated as deferral stock for purposes of subparagraph (B)(iii) if such individual (immediately after such purchase) holds any deferral stock with respect to which an election has been in effect under this subsection for a longer period than the election with respect to the stock so purchased.

(iii)Purchase of all outstanding deferral stock
The requirements of subclauses (I) and (II) of subparagraph (B)(iii) shall be treated as met if the stock so purchased includes all of the corporation’s outstanding deferral stock.

(iv)Reporting
Any corporation which has outstanding deferral stock as of the beginning of any calendar year and which purchases any of its outstanding stock during such calendar year shall include on its return of tax for the taxable year in which, or with which, such calendar year ends the total dollar amount of its outstanding stock so purchased during such calendar year and such other information as the Secretary requires for purposes of administering this paragraph.

(5)Controlled groups
For purposes of this subsection, all persons treated as a single employer under section 414(b) shall be treated as 1 corporation.

(6)Notice requirement
Any corporation which transfers qualified stock to a qualified employee shall, at the time that (or a reasonable period before) an amount attributable to such stock would (but for this subsection) first be includible in the gross income of such employee—
(A)certify to such employee that such stock is qualified stock, and
(B)notify such employee—
(i)that the employee may be eligible to elect to defer income on such stock under this subsection, and
(ii)that, if the employee makes such an election—
(I)the amount of income recognized at the end of the deferral period will be based on the value of the stock at the time at which the rights of the employee in such stock first become transferable or not subject to substantial risk of forfeiture, notwithstanding whether the value of the stock has declined during the deferral period,
(II)the amount of such income recognized at the end of the deferral period will be subject to withholding under section 3401(i) at the rate determined under section 3402(t), and
(III)the responsibilities of the employee (as determined by the Secretary under paragraph (3)(A)(ii)) with respect to such withholding.
(7)Restricted stock units
This section (other than this subsection), including any election under subsection (b), shall not apply to restricted stock units.
"""



######################################## 提示词元素 ########################################

##### 提示词元素 1: `user` 角色
# 确保您的消息API调用始终在消息数组中以 `user` 角色开始。
# 上面定义的 get_completion() 函数会自动为您执行此操作。

##### 提示词元素 2: 任务上下文
# 给模型关于它应该承担的角色或您希望它通过提示词承担的目标和总体任务的上下文。
# 最好在提示词正文的早期放置上下文。
TASK_CONTEXT = ""

##### 提示词元素 3: 语调上下文
# 如果对交互很重要，告诉模型应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = ""

##### 提示词元素 4: 要处理的输入数据
# 如果提示词中有模型需要处理的数据，请在此处包含在相关的XML标签内。
# 可以包含多个数据片段，但务必将每个数据封装在自己的XML标签集中。
# 根据任务，此元素可能不是必需的。顺序也是灵活的。
INPUT_DATA = ""

##### 提示词元素 5: 示例
# 为模型提供至少一个可以模仿的理想响应示例。将其封装在 <example></example> XML 标签中。可以提供多个示例。
# 如果您确实提供多个示例，请给模型关于这是什么示例的上下文，并将每个示例封装在自己的XML标签集中。
# 示例可能是知识工作中让模型按预期行为的最有效工具。
# 确保为模型提供常见边缘情况的示例。如果您的提示词使用草稿本，提供草稿本应该如何看起来的示例是有效的。
# 通常更多示例 = 更好。
EXAMPLES = ""

##### 提示词元素 6: 详细任务描述和规则
# 详细说明您希望模型执行的具体任务，以及模型可能需要遵循的任何规则。
# 这也是您可以在模型没有答案或不知道时给它一个"出路"的地方。
# 最好将此描述和规则展示给朋友，以确保它布局合理，任何模糊的词语都得到清晰定义。
TASK_DESCRIPTION = ""

##### 提示词元素 7: 立即任务描述或请求 #####
# "提醒"模型或准确告诉模型期望它立即做什么来完成提示词的任务。
# 这也是您放置额外变量（如用户问题）的地方。
# 向模型重申其立即任务通常不会有害。最好在长提示词末尾执行此操作。
# 这比在开头放置会产生更好的结果。
# 将用户查询放在提示词底部附近也是一般的良好做法。
IMMEDIATE_TASK = ""

##### 提示词元素 8: 预认知（逐步思考）
# 对于有多个步骤的任务，最好告诉模型在给出答案之前先逐步思考
# 有时，您可能甚至必须说"在您给出答案之前..."以确保模型首先执行此操作。
# 并非所有提示词都需要，但如果包含，最好在长提示词末尾且在最终立即任务请求或描述之后执行此操作。
PRECOGNITION = ""

##### 提示词元素 9: 输出格式
# 如果您希望模型的响应以特定方式格式化，请清楚地告诉模型该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果包含它，最好将其放在提示词末尾而不是开头。
OUTPUT_FORMATTING = ""

##### 提示词元素 10: 预填充模型的响应（如果有）
# 用一些预填充的词语开始模型的答案，以引导模型的行为或响应。
# 如果您想预填充模型的响应，您必须将其放在API调用的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示词
print("--------------------------- 包含变量替换的完整提示词 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
from hints import exercise_9_1_solution; print(exercise_9_1_solution)

### 练习 9.2 - 代码机器人
在这个练习中，我们将编写一个**编程辅助和教学机器人的提示词，它读取代码并在适当时提供指导性修正**。使用与描述和您在前面复杂提示词示例中看到的示例相匹配的内容填写提示词元素字段。一旦您填写完所有您想填写的提示词元素，运行单元格以查看连接的提示词以及模型的响应。

我们建议您阅读变量内容（在这种情况下是 `{CODE}`）以了解模型预期要处理的内容。确保在您的提示词中的某个地方直接引用 `{CODE}`（使用与其他示例中相同的f-string语法），以便实际的变量内容可以被替换进去。

In [ ]:
######################################## 输入变量 ########################################

# 输入变量 - 模型需要读取并协助用户修正的代码
CODE = """
# 打印乘法逆元的函数
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## 提示词元素 ########################################

##### 提示词元素 1: `user` 角色
# 确保您的消息API调用始终在消息数组中以 `user` 角色开始。
# 上面定义的 get_completion() 函数会自动为您执行此操作。

##### 提示词元素 2: 任务上下文
# 给模型关于它应该承担的角色或您希望它通过提示词承担的目标和总体任务的上下文。
# 最好在提示词正文的早期放置上下文。
TASK_CONTEXT = ""

##### 提示词元素 3: 语调上下文
# 如果对交互很重要，告诉模型应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = ""

##### 提示词元素 4: 详细任务描述和规则
# 详细说明您希望模型执行的具体任务，以及模型可能需要遵循的任何规则。
# 这也是您可以在模型没有答案或不知道时给它一个"出路"的地方。
# 最好将此描述和规则展示给朋友，以确保它布局合理，任何模糊的词语都得到清晰定义。
TASK_DESCRIPTION = ""

##### 提示词元素 5: 示例
# 为模型提供至少一个可以模仿的理想响应示例。将其封装在 <example></example> XML 标签中。可以提供多个示例。
# 如果您确实提供多个示例，请给模型关于这是什么示例的上下文，并将每个示例封装在自己的XML标签集中。
# 示例可能是知识工作中让模型按预期行为的最有效工具。
# 确保为模型提供常见边缘情况的示例。如果您的提示词使用草稿本，提供草稿本应该如何看起来的示例是有效的。
# 通常更多示例 = 更好。
EXAMPLES = ""

##### 提示词元素 6: 要处理的输入数据
# 如果提示词中有模型需要处理的数据，请在此处包含在相关的XML标签内。
# 可以包含多个数据片段，但务必将每个数据封装在自己的XML标签集中。
# 根据任务，此元素可能不是必需的。顺序也是灵活的。
INPUT_DATA = ""

##### 提示词元素 7: 立即任务描述或请求 #####
# "提醒"模型或准确告诉模型期望它立即做什么来完成提示词的任务。
# 这也是您放置额外变量（如用户问题）的地方。
# 向模型重申其立即任务通常不会有害。最好在长提示词末尾执行此操作。
# 这比在开头放置会产生更好的结果。
# 将用户查询放在提示词底部附近也是一般的良好做法。
IMMEDIATE_TASK = ""

##### 提示词元素 8: 预认知（逐步思考）
# 对于有多个步骤的任务，最好告诉模型在给出答案之前先逐步思考
# 有时，您可能甚至必须说"在您给出答案之前..."以确保模型首先执行此操作。
# 并非所有提示词都需要，但如果包含，最好在长提示词末尾且在最终立即任务请求或描述之后执行此操作。
PRECOGNITION = ""

##### 提示词元素 9: 输出格式
# 如果您希望模型的响应以特定方式格式化，请清楚地告诉模型该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果包含它，最好将其放在提示词末尾而不是开头。
OUTPUT_FORMATTING = ""

##### 提示词元素 10: 预填充模型的响应（如果有）
# 用一些预填充的词语开始模型的答案，以引导模型的行为或响应。
# 如果您想预填充模型的响应，您必须将其放在API调用的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示词
print("--------------------------- 包含变量替换的完整提示词 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
from hints import exercise_9_2_solution; print(exercise_9_2_solution)

---

## 恭喜您并展望下一步！

如果您完成了所有练习，**您现在已经成为了前0.1%的大语言模型训练师**。精英中的精英！

您学到的技术，从逐步思考到分配角色，从使用示例到全面清晰的写作，都可以**以无数种方式进行合并、重新组合和调整**。

提示词工程是一个非常新的学科，所以请保持开放的心态。您可能就是发现下一个伟大提示技巧的人。

如果您想看到**更多优秀提示词的示例**以获得灵感：			
- 从我们的[说明书](https://anthropic.com/cookbook)中学习生产就绪提示词的示例
- 阅读我们的[提示指南](https://docs.anthropic.com/claude/docs/prompt-engineering)
- 查看我们的[提示库](https://anthropic.com/prompts)以获得灵感
- 尝试我们的实验性[元提示词](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental)让Claude为您编写提示词模板！
- 在我们的[discord服务器](https://anthropic.com/discord)中提问
- 了解[Anthropic API参数](https://docs.anthropic.com/claude/reference/complete_post)如temperature和`max_tokens`
- 如果您想要学术化一些，阅读一些关于提示词工程的[论文](https://www.promptingguide.ai/papers)
- 练习构建提示词来让Claude做您感兴趣的事情

如果您想了解超出本教程范围的一些真正高级的提示技术，请点击进入附录！但首先，运行下面的单元格。

In [ ]:
# 提示词
PROMPT = "为一位刚刚完成提示词工程课程的出色学生写一首颂歌，以十四行诗的形式。"

# 打印模型的响应
print(get_completion(PROMPT))

---

## 示例练习场

这是一个供您自由试验本课程中展示的提示词示例的区域，您可以调整提示词来看看它如何影响模型的响应。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 对话历史（这也可以在API调用中作为之前的`user`和`assistant`消息添加）
HISTORY = """客户：请给我两个社会学专业可能的职业选择。

小智：以下是社会学专业的两个潜在职业：

- 社会工作者 - 社会学为理解人类行为和社会系统提供了坚实的基础。通过额外的培训或认证，社会学学位可以让毕业生胜任社会工作者、案例管理员、顾问和帮助个人和团体的社区组织者等角色。

- 人力资源专员 - 从社会学中获得的群体动力学和组织行为学理解适用于人力资源职业。毕业生可能在招聘、员工关系、培训和发展、多样性和包容性以及其他HR职能中找到角色。对社会结构和机构的关注也支持公共政策、非营利管理和教育等相关职业。"""

# 第二个输入变量 - 用户的问题
QUESTION = "这两个职业中哪一个需要超过学士学位的学历？"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "您将扮演由星途职业公司创建的名为小智的AI职业教练。您的目标是为用户提供职业建议。您将回复在星途网站上的用户，如果您不以小智的身份回应，他们会感到困惑。"

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "您应该保持友好的客户服务语调。"

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """以下是交互的一些重要规则：
- 始终保持角色，作为小智，星途职业公司的AI
- 如果您不确定如何回应，请说"抱歉，我没有理解。您能重新表述一下您的问题吗？"
- 如果有人问不相关的问题，请说"抱歉，我是小智，我提供职业建议。您今天有什么职业问题我可以帮助您吗？\""""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """以下是在标准交互中如何回应的示例：
<example>
客户：您好，您是如何被创建的，您做什么？
小智：您好！我的名字是小智，我由星途职业公司创建，提供职业建议。今天我可以为您提供什么帮助？
</example>"""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""以下是问题之前的对话历史（您和用户之间）。如果没有历史记录，可能为空：
<history>
{HISTORY}
</history>

以下是用户的问题：
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "您如何回应用户的问题？"

##### 提示词元素 8: 预认知（逐步思考）
# 对于有多个步骤的任务，最好告诉模型在给出答案之前先逐步思考
# 有时，您可能甚至必须说"在您给出答案之前..."以确保模型首先执行此操作。
# 并非所有提示词都需要，但如果包含，最好在长提示词末尾且在最终立即任务请求或描述之后执行此操作。
PRECOGNITION = "在回应之前先思考您的答案。"

##### 提示词元素 9: 输出格式
# 如果您希望模型的响应以特定方式格式化，请清楚地告诉模型该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果包含它，最好将其放在提示词末尾而不是开头。
OUTPUT_FORMATTING = "将您的响应放在 <response></response> 标签中。"

##### 提示词元素 10: 预填充模型的响应（如果有）
# 用一些预填充的词语开始模型的答案，以引导模型的行为或响应。
# 如果您想预填充模型的响应，您必须将其放在API调用的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = "[小智] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示词
print("--------------------------- 包含变量替换的完整提示词 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
######################################## INPUT VARIABLES ########################################

# 第一个输入变量 - 法律文档
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# 第二个输入变量 - 用户的问题
QUESTION = "是否有关于飓风期间如何处理宠物的法律？"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "您是一位专业律师。"

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""以下是已编译的一些研究资料。请使用它来回答用户的法律问题。
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """在您的答案中引用法律研究时，请使用包含搜索索引ID的方括号，后跟句号。将这些放在进行引用的句子末尾。正确引用格式的示例：

<examples>
<example>
此类犯罪的诉讼时效在10年后到期。[3]。
</example>
<example>
但是，当双方明确放弃时，保护不适用。[5]。
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """对以下问题写一个清晰、简洁的答案：

<question>
{QUESTION}
</question>

不应超过几段。如果可能，应以直接回答用户问题的单个句子结尾。但是，如果编译的研究中没有足够的信息来产生这样的答案，您可以推托并写"抱歉，我手头没有足够的信息来回答这个问题。\""""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "在回答之前，请在 <relevant_quotes> 标签中提取研究中最相关的引用。"

##### 提示词元素 9: 输出格式
# 如果您希望模型的响应以特定方式格式化，请清楚地告诉模型该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果包含它，最好将其放在提示词末尾而不是开头。
OUTPUT_FORMATTING = "将您的两段响应放在 <answer> 标签中。"

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示词
print("--------------------------- 包含变量替换的完整提示词 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))